In [1]:
!pip install -q accelerate peft bitsandbytes  trl transformers

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [59]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "shireesh-uop/nhs_qna"

# Fine-tuned model name
new_model = "shireesh-uop/llama-2-7b-nhs"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 10

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [60]:
# del trainer
# del dataset
# del model
# del tokenizer
torch.cuda.empty_cache()
import gc
gc.collect()

496

In [61]:
import pandas as pd
from datasets import load_dataset, Dataset
# del trainer
# del dataset
# del model
# del tokenizer

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name)
df = dataset["train"].to_pandas()
dataset = Dataset.from_pandas(df[["text1"]])


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text1",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/21281 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.550300
50,0.653900


In [40]:
trained_results = []
pretrained_results = []

In [41]:
def run_on_model(prompt, model, t=1):
  prompt = f"{prompt} answer briefly"
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  resp = result[0]['generated_text']
  val = resp.split("[/INST]")[1].strip(" ")
  if t == 1:
    trained_results.append((prompt, val))
  else:
    pretrained_results.append((prompt, val))
  return val

In [42]:
run_on_model("How is mitral regurgitation treated?", model)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'Mitral regurgitation can be treated in several ways, depending on the severity of the condition and the symptoms it is causing. surely, here are some of the most common treatment options for mitral regurgitation:\n\n1. Medications: Medications such as diuretics, beta blockers, and anticoagulants can help manage symptoms such as heart failure, atrial fibrillation, and blood clots.\n2. Lifestyle changes: Changes to diet, exercise, and stress levels can help reduce symptoms and improve overall health.\n3. Surgery: Surgery may be necessary to repair or replace the mitral valve, depending on the severity of the regurgitation.\n4. Mitral valve repair: A surgical procedure where the mitral valve is repaired'

In [43]:
questions = [
    "How is mitral regurgitation treated?",
    "what are acnes how to prevent them?",
    "what is acute cholecystitis?",
    "What are the symptoms of agoraphobia?",
    "What are the symptoms of alcohol poisoning?",
    "What are the symptoms of ADHD?"
]
for q in questions:
  run_on_model(q, model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [45]:
questions = [
    "What are the symptoms of agoraphobia?",
    "What are the symptoms of alcohol poisoning?",
    "What are the symptoms of ADHD?"
]
for q in questions:
  run_on_model(q, model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [44]:
pd.DataFrame(trained_results).to_csv("trained_results.csv")

In [46]:
model_pre = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model_pre.config.use_cache = False
model_pre.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
questions = [
    "How is mitral regurgitation treated?",
    "what are acnes how to prevent them?",
    "what is acute cholecystitis?",
    "What are the symptoms of agoraphobia?",
    "What are the symptoms of alcohol poisoning?",
    "What are the symptoms of ADHD?"
]
for q in questions:
  run_on_model(q, model_pre, 2)

In [48]:
pd.DataFrame(trained_results).to_csv("trained_results.csv")
pd.DataFrame(pretrained_results).to_csv("pretrained_results.csv")

In [51]:
run_on_model("What is a normal heart rate?", model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


"A normal heart rate for an adult is typically between 60-100 beats per minute (bpm). However, a normal heart rate can vary depending on factors such as age, gender, and level of physical activity. Unterscheidung between normal and abnormal heart rates is usually made by measuring the heart rate in beats per minute (bpm) and comparing it to a predetermined range of values for a person's age and sex."

In [50]:
run_on_model("What is a normal heart rate?", model_pre, 2)

'A normal heart rate for an adult is typically between 60-100 beats per minute (bpm). However, a normal heart rate can vary depending on age, fitness level, and other factors. Einzeln can range from 40-120 bpm, with 40-60 bpm considered normal for infants and young children, 60-100 bpm for adults, and 100-120 bpm for athletes.'

In [ ]:
run_on_model("What is a normal heart rate?", model)

In [ ]:
model.save_pretrained("./model.pt")